<a href="https://colab.research.google.com/github/rizalpernata1/Temperature_Forecasting_with_LSTM/blob/main/Temperature_Forecasting_with_LSTM_Jena_Climate_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

## Load data

In [2]:
df = pd.read_csv('/content/jena_climate_2009_2016.csv')

## Preprocessing the data

Performing data preprocessing steps, such as converting the date format, setting the index, and filling missing values.

In [3]:
df['Date Time'] = pd.to_datetime(df['Date Time'], format='%d.%m.%Y %H:%M:%S')
df = df.set_index('Date Time')
df = df.resample('H').mean().fillna(method='ffill')  # Resample to hourly data and forward fill missing values

## Selecting the variable and scaling the data:

Selecting the variable of interest (e.g., temperature) and scaling the data using the MinMaxScaler to normalize it.

In [4]:
data = df['T (degC)'].values.reshape(-1, 1)

In [5]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

## Splitting the data

Splitting the dataset into training and testing data.

In [6]:
train_size = int(len(data_scaled) * 0.8)
train_data = data_scaled[:train_size]
test_data = data_scaled[train_size:]

## Data preparation

Defining a function to prepare the data in the form of sequences with the given time steps.

In [7]:
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)-n_steps-1):
        X.append(data[i:(i+n_steps), 0])
        y.append(data[i+n_steps, 0])
    return np.array(X), np.array(y)

n_steps = 24  # Number of time steps to consider
X_train, y_train = prepare_data(train_data, n_steps)
X_test, y_test = prepare_data(test_data, n_steps)

#### Reshape input data to be 3-dimensional in the form [samples, time steps, features]

In [8]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

## Building the LSTM model

Building the LSTM model using the Sequential API from Keras, adding LSTM and Dense layers.

In [9]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

## Training and Evaluate the model

Training the model using the training data.

Evaluating the performance of the model using both the training and testing data.

In [10]:
model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1)

Epoch 1/50
3505/3505 [==============================] - 51s 14ms/step - loss: 0.0025
Epoch 2/50
3505/3505 [==============================] - 41s 12ms/step - loss: 1.8893e-04
Epoch 3/50
3505/3505 [==============================] - 42s 12ms/step - loss: 1.3538e-04
Epoch 4/50
3505/3505 [==============================] - 41s 12ms/step - loss: 1.2700e-04
Epoch 5/50
3505/3505 [==============================] - 43s 12ms/step - loss: 1.2445e-04
Epoch 6/50
3505/3505 [==============================] - 41s 12ms/step - loss: 1.2023e-04
Epoch 7/50
3505/3505 [==============================] - 42s 12ms/step - loss: 1.1946e-04
Epoch 8/50
3505/3505 [==============================] - 41s 12ms/step - loss: 1.1633e-04
Epoch 9/50
3505/3505 [==============================] - 42s 12ms/step - loss: 1.1474e-04
Epoch 10/50
3505/3505 [==============================] - 42s 12ms/step - loss: 1.1557e-04
Epoch 11/50
3505/3505 [==============================] - 42s 12ms/step - loss: 1.1298e-04
Epoch 12/50
3505/3505 [

In [11]:
train_loss = model.evaluate(X_train, y_train, verbose=0)
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Training loss: {train_loss:.4f}')
print(f'Testing loss: {test_loss:.4f}')

Training loss: 0.0001
Testing loss: 0.0001


## Making predictions

Making predictions on the testing data using the trained model.

In [12]:
predictions = model.predict(X_test)

438/438 [==============================] - 2s 4ms/step


## Scaling back and Evaluating the performance

Reverting the scaled predictions back to the original scale using inverse transformation.

Calculating the Root Mean Squared Error (RMSE) to measure the prediction error of the model.

In [13]:
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

In [14]:
rmse = np.sqrt(np.mean((predictions - y_test) ** 2))
print(f'Root Mean Squared Error: {rmse:.4f}')

Root Mean Squared Error: 0.5741
